<a id="toc"></a>
# DEIMOS beamline simulation with PyOptiX
***

Contents :
1. [Optical elements declaration](#def_opt)
1. [Definition of optical parameters](#def_param)
1. [Alignment scripts](#alignement)
1. [Simulation execution](#exec)
1. [Calculations](#calcu)
1. [Monochromator resolution](#monores)

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '21/OCT/2024'
__changed__ = '13/NOV/2024'


import ctypes

import barc4xoc.beam as bm
import numpy as np
import pandas as pd
import pyoptix
import pyoptix.classes as opx
from barc4xoc.aux_pyoptix import save_beam_to_csv, save_resolution_curve
from barc4xoc.misc import coddington_equations
from scipy.constants import c, degree, eV, h, micro, milli, nano, pi, pico

hc = h*c/eV

pyoptix.set_aperture_active(False)
pyoptix.output_notebook()


<a id="def_opt"></a>
## Optical elements declaration
[Back to the top](#toc)

In [ ]:
Deimos = opx.Beamline(name="Deimos - upgrade")

# ------------------
# source
# ------------------
undulator_SII = opx.UndulatorSource(name="SII")

# ------------------
# optical elements
# ------------------
pupil = opx.PlaneFilm(name="pupil")

# M1
m1a = opx.PlaneMirror(name="M1A")
m1b = opx.ToroidalMirror(name="M1B")
m1c = opx.ToroidalMirror(name="M1C")

# monochromator
# grating_1600 = opx.PlaneHoloGrating(name="grating_1600")   # VGD
grating_1600 = opx.PlaneGrating(name="grating_1600")       
grating_2400 = opx.PlaneGrating(name="grating_2400")       # MCA

m2 = opx.PlaneMirror(name="M2")
m3 = opx.CylindricalMirror(name="M3")
mono_exit_slit = opx.PlaneFilm(name="mono_foc_ver")

# M4
m4a = opx.ConicCylindricalMirror(name="M4A")
m4b = opx.ConicCylindricalMirror(name="M4B")

# M5
m5a = opx.ConicCylindricalMirror(name="M5A")
m5b = opx.ConicCylindricalMirror(name="M5B")

# ------------------
# endstations
# ------------------
cromag = opx.PlaneFilm(name="cromag")
mk2t = opx.PlaneFilm(name="mk2t")

In [ ]:
undulators = [undulator_SII]
m1 = [m1b, m1c]
gratings = [grating_1600, grating_2400]
wolters = [[m4a, m5a], [m4b, m5b], []]
endstations = [cromag, mk2t]

for undulator in undulators:
    for m1bis in m1:
        for grating in gratings:
            beamline = [undulator, pupil, m1a, m1bis, grating, m2, m3, mono_exit_slit]           
   
            for focusing in wolters:
                if len(focusing) > 1:
                    chain_name = f"{undulator.name}_{m1bis.name}_G{grating.name.split('_')[1]}_WLT_{focusing[-2].name[-1]}"
                    Deimos.chains[chain_name] = beamline + focusing + endstations
                else:
                    chain_name = f"{undulator.name}_{m1bis.name}_G{grating.name.split('_')[1]}_mono"
                    Deimos.chains[chain_name] = beamline


In [ ]:
print("Low energy configurations:\n")
for chain_name in Deimos.chains:    
    if "M1B" in chain_name:
        print(chain_name,":\n\t",Deimos.chains[chain_name])

In [ ]:
print("High energy configurations:\n")
for chain_name in Deimos.chains:    
    if "M1C" in chain_name:
        print(chain_name,":\n\t",Deimos.chains[chain_name])

<a id="def_param"></a>
## Definition of optical parameters

In this section we define the static parameters of the optical elements. Characteristics that change with 
either configuration or energy are defined in the [Alignment scripts](#alignement) section.

[Back to the top](#toc)

### SOLEIL-II lattice (HU52 undulator)

In [ ]:
e_beam = opx.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.0906/100, current=0.500,
                  emittance_x=84.55*pico, emittance_y=25.36*pico,
                  beta_x=1.59, eta_x=0, etap_x=0, alpha_x=0,
                  beta_y=1.51, eta_y=0, etap_y=0, alpha_y=0)
e_beam.print_rms()

hu52 = opx.MagneticStructure(period_length=52.4e-3, number_of_periods=30)

undulator_SII.electron_beam = e_beam
undulator_SII.magnetic_structure = hu52

undulator_SII.write_syned_config(r".\resources\oasys_soleil-II_hu52","Soleil-II - HU52")

#### Alignment script

In [ ]:
def align_undulator(active_chain, wavelength, **kwargs):
    """
    Sets photon beam size and divergence as a function of wavelength in [m]
    """

    verbose = kwargs.get("verbose", False)
    dist_slit = kwargs.get("dist_slit", 21)
    
    if verbose:
        print("\n>>>> Aligning undulator")

    active_chain[0].set_undulator(wavelength, **kwargs)
    active_chain[1].distance_from_previous = dist_slit

### Entrance pupil
[Back to the top](#toc)

In [ ]:
pupil.distance_from_previous = 21
pupil.recording_mode = opx.RecordingMode.recording_output
pupil.next = m1a

### M1

[Back to the top](#toc)

In [ ]:
old_m1_grazing_angles = [2.53*degree, 1.19*degree]

old_m1_distances = opx.M1_triad_distances(incident_angle=old_m1_grazing_angles,
                                          transverse_distance=41e-3, 
                                          verbose=1)

In [ ]:
m1_grazing_angles = [1.75*degree, 0.75*degree]

m1_distances = opx.M1_triad_distances(incident_angle=m1_grazing_angles,
                                      transverse_distance=26e-3, 
                                      verbose=1)

#### - M1A

In [ ]:
m1a.distance_from_previous = 0
m1a.phi = -90*degree # rad
m1a.recording_mode = opx.RecordingMode.recording_output

#### - M1B

In [ ]:
p = pupil.distance_from_previous + m1_distances[0][0]
qminor = 1E23
qmajor = (m1_distances[1][1]-m1_distances[0][1]) + 5 + 0.7 + 4 # we focus on the mochromator slit

rminor = coddington_equations(m1_grazing_angles[0]/degree, p=p, q=qminor)["R"][1]
rmajor = coddington_equations(m1_grazing_angles[0]/degree, p=p, q=qmajor)["R"][0]

print("Toroidal mirror with side-bounce reflection:")
print(f"- source at {p:.3e} m")
print(f"- (sagittal) image at {qminor:.3} m")
print(f"- (meridional/tangential) image at {qmajor:.3e} m")

print("\nApplying the Coddington equations:")
print(f"- minor (sagittal) radius {rminor:.3e} m")
print(f"- major (meridional/tangential) radius {rmajor:.3e} m")

m1b.distance_from_previous = m1_distances[0][0]
m1b.theta = m1_grazing_angles[0] # rad
m1b.phi = 180*degree # rad
m1b.minor_curvature = 1/rminor # m-1   # m1b.minor_curvature = 1/1.894 # m-1
m1b.major_curvature = 1/rmajor # m-1   # m1b.major_curvature = 1/228.0 # m-1
m1b.recording_mode = opx.RecordingMode.recording_output
# m1b.next = grating_1600

#### - M1C

In [ ]:
p = pupil.distance_from_previous + m1_distances[1][0]
qminor = 1E23
qmajor = 5 + 0.7 + 4 # we focus on the mochromator slit

rminor = coddington_equations(m1_grazing_angles[1]/degree, p=p, q=qminor)["R"][1]
rmajor = coddington_equations(m1_grazing_angles[1]/degree, p=p, q=qmajor)["R"][0]

print("Toroidal mirror with side-bounce reflection:")
print(f"- source at {p:.3e} m")
print(f"- (sagittal) image at {qminor:.3e} m")
print(f"- (meridional/tangential) image at {qmajor:.3e} m")

print("\nApplying the Coddington equations:")
print(f"- minor (sagittal) radius {rminor:.3e} m")
print(f"- major (meridional/tangential) radius {rmajor:.3e} m")

m1c.distance_from_previous = m1_distances[1][0]
m1c.theta = m1_grazing_angles[1] # rad
m1c.phi = 180*degree # rad
m1c.minor_curvature = 1/rminor   # m-1   # m1c.minor_curvature = 1/0.91  # m-1
m1c.major_curvature = 1/rmajor   # m-1   # m1c.major_curvature = 1/455   # m-1
m1c.recording_mode = opx.RecordingMode.recording_output
# m1b.next = grating_2400

#### Alignment script

In [ ]:
def align_m1(active_chain, m1_distances, **kwargs):
    """
    Sets the M1A angle and distance from mono first optical element to M1bis
    """
    
    dist_next = kwargs.get("dist_next", 5)
    verbose = kwargs.get("verbose", False)

    if verbose:
        print("\n>>>> Aligning M1")
   
    active_chain[2].theta = active_chain[3].theta
    active_chain[2].next  = active_chain[3]
    active_chain[3].next = active_chain[4]
    active_chain[4].distance_from_previous = dist_next

    if "1B" in active_chain[3].name:
        active_chain[4].distance_from_previous += (m1_distances[1][1]-m1_distances[0][1])       

    if verbose:
        name = active_chain[3].name
        angle = active_chain[3].theta/degree
        dist = active_chain[3].distance_from_previous
        print(f"M1A-{name} alignment: grazing angle {angle:.2f} degrees and {dist:.3f} m between them")

### Monochromator

[Back to the top](#toc)

#### Grating 1600 l/mm

In [ ]:
# grating_1600.distance_from_previous = 5.0
# grating_1600.line_density = 1600/milli
# grating_1600.from_solemio(1e-10, 1e-10, 0.7660444, -0.56176)     # RC20240502 - where did you get this?
# grating_1600.order_align = 1 
# grating_1600.order_use = 1 
# grating_1600.phi = -90*degree
# grating_1600.recording_mode = opx.RecordingMode.recording_output
# grating_1600.next = m2
# grating_1600.show_vls_law(300e-3,3)

grating_1600.distance_from_previous = 5.0
grating_1600.line_density = 1600/milli
grating_1600.phi = -90*degree
grating_1600.order_align = 1
grating_1600.order_use = 1
grating_1600.recording_mode = opx.RecordingMode.recording_output
grating_1600.next = m2

#### Grating 2400 l/mm

In [ ]:
grating_2400.distance_from_previous = 5.0
grating_2400.line_density = 2400/milli
grating_2400.phi = -90*degree
grating_2400.order_align = 1
grating_2400.order_use = 1
grating_2400.recording_mode = opx.RecordingMode.recording_output
grating_2400.next = m2

#### M2

In [ ]:
m2.phi = 180*degree
m2.recording_mode = opx.RecordingMode.recording_output
m2.next = m3

#### M3 - vertical/horizontal deflection

In [ ]:
m3.theta = 0.44*degree
m3.recording_mode = opx.RecordingMode.recording_output
m3.next = mono_exit_slit

#### Alignment script

In [ ]:
def align_m3(active_chain, **kwargs):

    m3_orientation = kwargs.get("m3_orientation", 'horizontal')
    verbose = kwargs.get("verbose", False)

    if verbose:
        print("\n>>>> Aligning M3")

    if m3_orientation in ['ver', 'vertical', 'up']:
        vertical = True
    elif m3_orientation in ['hor', 'horizontal', 'side']:
        vertical = False
    else:
        raise ValueError("No orientation given to M3") 

    for count, oe in enumerate(active_chain):
        if "m3" in oe.name.lower():
            mirror = oe
            mirror_oe_pos = count

    p = 1E23
    theta = mirror.theta

    if vertical:
        qminor, qmajor = 1E23, active_chain[mirror_oe_pos+1].distance_from_previous
        phi, curvature_radius, axis_angle = 180 * degree, "major", 0 * degree
        phi_mes = 0
        reflection_type = "bounce-up"
    else:
        qminor, qmajor = active_chain[mirror_oe_pos+1].distance_from_previous, 1E23
        phi, curvature_radius, axis_angle = 90 * degree, "minor", 90 * degree
        phi_mes = phi
        reflection_type = "side-bounce"

    rminor = coddington_equations(theta/degree, p=p, q=qminor)["R"][1]
    rmajor = coddington_equations(theta/degree, p=p, q=qmajor)["R"][0]

    if verbose:
        print(f"Toroidal mirror with {reflection_type} reflection:")
        print(f"- source at {p:.3e} m")
        print(f"- (sagittal) image at {qminor:.3e} m")
        print(f"- (meridional/tangential) image at {qmajor:.3e} m")
        print("\nApplying the Coddington equations:")
        print(f"- minor (sagittal) radius {rminor:.3e} m")
        print(f"- major (meridional/tangential) radius {rmajor:.3e} m")

    mirror.phi = phi
    mirror.curvature = 1 / (rmajor if curvature_radius == "major" else rminor)
    mirror.axis_angle = axis_angle
    active_chain[mirror_oe_pos+1].phi = phi_mes


#### Mono exit slits

In [ ]:
mono_exit_slit.distance_from_previous = 3.8 + 0.2
# mono_exit_slit.add_rectangular_stop(20*micro, 20*micro, opacity=0)
mono_exit_slit.recording_mode = opx.RecordingMode.recording_output

#### Alignment script

In [ ]:
def align_mono(active_chain, wavelength, alignment_condition, alignment_condition_value,
               GM2_trans_dist, GM3_proj_dist, **kwargs):
    """
    Sets the grating and M2 angles and relative distances between G/M2/M3
    """

    verbose = kwargs.get("verbose", False)

    grating = None
    for count, oe in enumerate(active_chain):
        if "reseau" in oe.name.lower() or "grating" in oe.name.lower():
            grating = oe
            mono_oe_pos = count

    if grating is None:
        raise ValueError("No grating appears in this beamline configuration")
    
    gdict = opx.align_grating(grating, verbose=0, 
                             apply_alignment=True, 
                             return_parameters=True,
                             condition=alignment_condition, 
                             condition_value=alignment_condition_value, 
                             lambda_align=wavelength, 
                             order=grating.order_align,
                             line_density=grating.line_density)
    
    gm2 = GM2_trans_dist/np.sin(gdict["deviation"])
    m2m3 = GM3_proj_dist-(GM2_trans_dist/np.tan(gdict["deviation"]))

    active_chain[int(mono_oe_pos+1)].theta = gdict["deviation"]/2
    active_chain[int(mono_oe_pos+1)].distance_from_previous = gm2 

    active_chain[int(mono_oe_pos+2)].distance_from_previous = m2m3

    if verbose:
        if alignment_condition == 'omega':
            alignment_condition_str =f"{alignment_condition_value/degree :.3f}"
        else:
            alignment_condition_str =alignment_condition_value
        print(f"\n>>>> {grating.name} grating alignment for "+
              f"{alignment_condition} {alignment_condition_str} - wavelength " +
              f"{wavelength/nano:.3f} nm (E={hc/wavelength:.3f} eV)")
        print(f"> alpha {gdict['alpha_deg']:.6f} deg")
        print(f"> beta {gdict['beta_deg']:.6f} deg")
        if alignment_condition == 'omega':
            cff_value = np.sin(gdict['beta'])/np.sin(gdict['alpha'])
            print(f">> cff {cff_value:.2f}")
        print(f"> G-M2 distance {gm2:.6f} m")
        print(f"> theta_m2 {(gdict['alpha_deg']+gdict['beta_deg'])/2:.6f} deg")
        print(f"> M2-M3 distance {m2m3:.6f} m")

### Beamline alignment procedure

In [ ]:
def align_call(alignment_wvl, emission_wvl, alignment_condition, alignment_condition_value, 
               m1_distances, GM2_trans_dist, GM3_proj_dist, **kwargs):

    align_undulator(Deimos.active_chain, emission_wvl, **kwargs)
    align_m1(Deimos.active_chain, m1_distances, **kwargs)
    align_mono(Deimos.active_chain, alignment_wvl, alignment_condition, 
               alignment_condition_value, GM2_trans_dist, GM3_proj_dist, **kwargs)
    align_m3(Deimos.active_chain, **kwargs)

Deimos.align_steps = align_call

def set_bl(energy_alignment, alignment_condition, alignment_condition_value, 
               m1_distances=m1_distances, GM2_trans_dist=20e-3, GM3_proj_dist=700e-3, **kwargs):

    energy_radiate =  kwargs.get("energy_radiate", energy_alignment)
    dE =  kwargs.get("dE", 0)
    nrays = kwargs.get("nrays", 500)

    alignment_wvl = hc/energy_alignment
    emission_wvl = hc/energy_radiate

    Deimos.align(alignment_wvl, emission_wvl, 
                 alignment_condition=alignment_condition, 
                 alignment_condition_value=alignment_condition_value, 
                 m1_distances=m1_distances, GM2_trans_dist=GM2_trans_dist,
                 GM3_proj_dist=GM3_proj_dist, **kwargs)

    Deimos.clear_impacts(clear_source=True)
    Deimos.active_chain[0].nrays = nrays
    if dE==0:
        Deimos.generate(emission_wvl)
    else:
        for E in np.arange(energy_radiate*(1-dE), energy_radiate*(1+dE)):
            Deimos.generate(hc/E)
    Deimos.radiate()

### M1 and mono design test

In [ ]:
nrays = 20000
verbose = False
rt_fldr = './results/pyoptix/mono_II/'

In [ ]:
Deimos.active_chain = "SII_M1B_G1600_mono"

In [ ]:
align_m3(Deimos.active_chain, m3_orientation='vertical', verbose=True)

In [ ]:
align_m3(Deimos.active_chain, m3_orientation='horizontal', verbose=True)

In [ ]:
E = 350
configs = ["SII_M1B_G1600_mono", "SII_M1C_G1600_mono", "SII_M1B_G2400_mono"]
m3_orientations = ['horizontal', 'vertical']
for config in configs:
    print(f"\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> {config}")

    Deimos.active_chain = config
    alignment = "cff" if 'G1600' in Deimos.active_chain_name else "omega"
    alignment_value = 0.2 if alignment == "cff" else 0.692 * degree

    for orient in m3_orientations:
        set_bl(E, alignment_condition=alignment, alignment_condition_value=alignment_value, 
            verbose=verbose, m3_orientation=orient, nrays=nrays)
        # Deimos.show_active_chain_orientation()
        oe = mono_exit_slit
        stats = bm.get_beam_stats(save_beam_to_csv(oe.get_impacts(), None), verbose=verbose)
        f_name = rt_fldr + f"{Deimos.active_chain_name}_@_{oe.name}_m3_{orient}_E{E}eV_{nrays:.0f}_rays.csv"
        beam = save_beam_to_csv(oe.get_impacts(), f_name)

### Focusing optics

[Back to the top](#toc)

In order to place the focusing system we fix the distance from mono exit slit (secondary source) and the image plane. This distance,
throw, is of $10$ m. We also set as $0.5$ m the distance between the centre of first and second focusing mirrors (M4 and M5). The distance between the first and second experimental stations (cromag and mk2T) is of $2$ m.

In [ ]:
throw = 10
sep_KB = 0.5
sep_exp_stations = 2.0

#### KB-A

##### M4A (KB horizontal focusing)

In [ ]:
m4a.theta = 0.44*degree
m4a.phi = -90*degree
m4a.theta0 = m4a.theta
m4a.recording_mode = opx.RecordingMode.recording_output
m4a.next = m5a

##### M5A (KB - vertical focusing)

In [ ]:
m5a.distance_from_previous = sep_KB
m5a.theta = 0.44*degree
m5a.phi = -90*degree
m5a.theta0 = m5a.theta
m5a.recording_mode = opx.RecordingMode.recording_output
m5a.next = cromag

#### KB-B

##### M4B (KB - horizontal focusing)

In [ ]:
m4b.theta = 0.44*degree
m4b.phi = -90*degree
m4b.theta0 = m4b.theta
m4b.recording_mode = opx.RecordingMode.recording_output
m4b.next = m5b

##### M5A (KB - vertical focusing)

In [ ]:
m5b.distance_from_previous = sep_KB
m5b.theta = 0.44*degree
m5b.phi = -90*degree
m5b.theta0 = m5b.theta
m5b.recording_mode = opx.RecordingMode.recording_output
m5b.next = cromag

#### Endstations

[Back to the top](#toc)

##### CroMag

In [ ]:
cromag.phi = 180*degree
cromag.recording_mode = opx.RecordingMode.recording_output
cromag.next = mk2t

##### mK2T

In [ ]:
mk2t.distance_from_previous = sep_exp_stations
mk2t.phi = 0
mk2t.recording_mode = opx.RecordingMode.recording_output

### Alignment script

In [ ]:
def align_kb(active_chain, p, q, **kwargs):

    verbose = True#kwargs.get("verbose", False)

    for count, oe in enumerate(active_chain):
        if "m4" in oe.name.lower():
            mirror_oe_pos = count
        
    print(mirror_oe_pos)
    dKB = (active_chain[mirror_oe_pos+1].distance_from_previous)/2
    print(active_chain[mirror_oe_pos+1].name)
    if verbose:
        print("\n> First mirror")
        print(f"- source at {(p-dKB):.3e} m")
        print(f"- image at  {(q+dKB):.3e} m")
        print(f"--   throw {(p+q):.3e} m")

        print("> Second mirror")
        print(f"- source at {(p+dKB):.3e} m")
        print(f"-  image at {(q-dKB):.3e} m")
        print(f"--   throw {(p+q):.3e} m")

        print(f"> Distance between mirrors is {dKB:.3e} m")

    print(p-dKB, p+dKB)
    print(q+dKB, q-dKB)

    # FIRST MIRROR
    active_chain[mirror_oe_pos].distance_from_previous = (p-dKB)
    active_chain[mirror_oe_pos].inverse_p = -1/(p-dKB)
    active_chain[mirror_oe_pos].inverse_q = 1/(q+dKB)

    # SECOND MIRROR
    active_chain[mirror_oe_pos+1].inverse_p = -1/(p+dKB)
    active_chain[mirror_oe_pos+1].inverse_q = 1/(q-dKB)

    active_chain[mirror_oe_pos+2].distance_from_previous = (q-dKB)

### Beamline alignment procedure

In [ ]:
def align_call(alignment_wvl, emission_wvl, alignment_condition, alignment_condition_value, 
               m1_distances, GM2_trans_dist, GM3_proj_dist, **kwargs):

    align_undulator(Deimos.active_chain, emission_wvl, **kwargs)
    align_m1(Deimos.active_chain, m1_distances, **kwargs)
    align_mono(Deimos.active_chain, alignment_wvl, alignment_condition, 
               alignment_condition_value, GM2_trans_dist, GM3_proj_dist, **kwargs)
    align_m3(Deimos.active_chain, **kwargs)
    align_kb(Deimos.active_chain, **kwargs)

Deimos.align_steps = align_call

def set_bl(energy_alignment, alignment_condition, alignment_condition_value, 
               m1_distances=m1_distances, GM2_trans_dist=20e-3, GM3_proj_dist=700e-3, **kwargs):

    energy_radiate =  kwargs.get("energy_radiate", energy_alignment)
    dE =  kwargs.get("dE", 0)
    nrays = kwargs.get("nrays", 500)

    alignment_wvl = hc/energy_alignment
    emission_wvl = hc/energy_radiate

    Deimos.align(alignment_wvl, emission_wvl, 
                 alignment_condition=alignment_condition, 
                 alignment_condition_value=alignment_condition_value, 
                 m1_distances=m1_distances, GM2_trans_dist=GM2_trans_dist,
                 GM3_proj_dist=GM3_proj_dist, **kwargs)

    Deimos.clear_impacts(clear_source=True)
    Deimos.active_chain[0].nrays = nrays
    if dE==0:
        Deimos.generate(emission_wvl)
    else:
        for E in np.arange(energy_radiate*(1-dE), energy_radiate*(1+dE)):
            Deimos.generate(hc/E)
    Deimos.radiate()

### KB design test

In [ ]:
nrays = 5000
verbose = False
rt_fldr = './results/pyoptix/sample_II/'

In [ ]:
Deimos.active_chain = "SII_M1B_G1600_WLT_A"

alignment = "cff" if 'G1600' in Deimos.active_chain_name else "omega"
alignment_value = 0.2 if alignment == "cff" else 0.692 * degree

set_bl(E, alignment_condition=alignment, alignment_condition_value=alignment_value, 
      verbose=verbose, nrays=nrays, p=6, q=3, m3_orientation='vertical')

oe = cromag
stats = bm.get_beam_stats(save_beam_to_csv(oe.get_impacts(), None), verbose=True)
# Deimos.show_active_chain_orientation()

<a id="exec"></a>
## Simulation execution
[Back to the top](#toc)

### Available beamline configurations

In [ ]:
print("Low energy configurations:")
for chain_name in Deimos.chains:    
    if "M1B" in chain_name:
        print(chain_name,":\n\t",Deimos.chains[chain_name])

In [ ]:
print("High energy configurations:")
for chain_name in Deimos.chains:    
    if "M1C" in chain_name:
        print(chain_name,":\n\t",Deimos.chains[chain_name])

The beamline can be conviniently simulated by selecting an **active chain** and an energy:

```python
Deimos.active_chain = "U52_M1B_G1600_mono"
set_bl(350, alignment_condition="cff", alignment_condition_value=0.2, verbose=True)
```

To check the orientation of the optical elements, draw the beamline:

```python
Deimos.show_active_chain_orientation()
Deimos.draw_active_chain()
spots = Deimos.draw_to_scale()
```

<a id="calcu"></a>
# Calculations
[Back to the top](#toc)

In [ ]:
rays = 5000
E = 350

<a id="monores"></a>
## Monochromator resolution 
[Back to the top](#toc)